# コロプレス図　＋　ツールチップ（ Tooltip ）

In [1]:
'''
geojsonから
　境界の座標データに変換：boundaries
　境界データのidを抽出:b_ids
 
情報元
　https://github.com/dataofjapan/land/blob/master/japan.geojson
'''
import pandas as pd
import geopandas as gpd
import shapely

df = gpd.read_file('data_geo/japan.geojson')

def get_boundary(df, idx):
    
    df_temp = df[df['id']==idx]
    boundary = []

    for _, row in df_temp.iterrows():
        polygons = row['geometry']

        if isinstance(polygons, shapely.geometry.multipolygon.MultiPolygon):
            polygons = [x for x in polygons]  # MultiPolygon に含まれる Polygon を取り出す
        elif isinstance(polygons, shapely.geometry.polygon.Polygon):
            polygons = [polygons]
        else:
            raise ValueError

        for p in polygons: 
            boundary.append(list(p.exterior.coords))
    return boundary

b_ids = pd.unique(df['id'])

boundaries = []

for b_id in b_ids:
    boundaries.append(get_boundary(df, b_id))

print("データ数（都道府県数）：", len(boundaries))
df.head()

データ数（都道府県数）： 47


,nam,nam_ja,id,geometry
0,Kyoto Fu,京都府,26,"(POLYGON ((135.036697387695 35.53733444213869,..."
1,Saga Ken,佐賀県,41,"(POLYGON ((129.862197875977 33.58753204345703,..."
2,Kumamoto Ken,熊本県,43,"(POLYGON ((131.110397338867 33.18313217163091,..."
3,Kagawa Ken,香川県,37,"(POLYGON ((134.332000732422 34.56313323974609,..."
4,Aichi Ken,愛知県,23,"(POLYGON ((136.960906982422 35.41453170776371,..."


In [2]:
'''
CSVデータ（data/data.csv）からデータフレームに読込:df_src
　id：インデックス
　pref：都道府県名
　pol：県庁所在地
　rad：自然放射線量（単位：μSv/h）
'''
import pandas as pd

df_src = pd.read_csv('data_geo/data.csv', index_col=0)
#df_src = pd.read_csv('data/data.csv', encoding='cp932', dtype='object', index_col=0)

df_src['index'] = df_src.index
df_src['rad'] = df_src['rad'].astype('float64')

display(df_src.head())
display(df_src.dtypes)


,pref,pol,rad,index
id,,,,
1,北海道,札幌,0.029,1
2,青森県,青森,0.030,2
3,岩手県,盛岡,0.027,3
4,宮城県,仙台,0.038,4
5,秋田県,秋田,0.036,5


pref      object
pol       object
rad      float64
index      int64
dtype: object

In [4]:
'''
STEP1:コロプレス図を描く

STEP2:ツールチップ(Tooltip)機能を追加する
  2-1:県境界を多角形(Polygon)で描く
  2-2:ツールチップのメッセージを設定する
'''
import folium
from IPython.display import IFrame

# マップオブジェクトの生成
center = [38, 137]
map_JPN = folium.Map(location=center, tiles='cartodbpositron', zoom_start=6)


# STEP1:コロプレス図を描く
folium.Choropleth(
    geo_data = 'data_geo/japan.geojson',
    data = df_src,
    columns = ['index', 'rad'],
    key_on = 'feature.properties.id',
    fill_color = 'YlOrRd'
).add_to(map_JPN)


#STEP2:ツールチップ(Tooltip)機能を追加する
msg = "<h4><center>{}<BR>( {} )<BR><BR>{:.3f} μSv/h</center></h4>"

for b_id, boundary in zip(b_ids, boundaries):
    def highlight_function(feature):
        return {
            'fillOpacity': 1,
            'fillColor': 'lightblue',
        }


    def style_function(feature):
        return {
            'weight': 1,
            'fillOpacity': 0,
            'Opacity': 1,
            'fillColor': None,
            'color': 'lightblue'
        }
    
    # 2-1:県境界を多角形(Polygon)で描く
    pref = folium.GeoJson(
        data={
            'type': 'Polygon',
            'coordinates': boundary
        },
        style_function = style_function,
        highlight_function=highlight_function
        )
    
    # 2-2:ツールチップのメッセージを設定する
    try:
        name1 = df_src.loc[b_id, 'pref']
        name2 = df_src.loc[b_id, 'pol']
        m_value = df_src.loc[b_id, 'rad']
    except:
        name1 = "None"
        name2 = "None"
        m_value = 0    
    pref.add_child(folium.Tooltip(msg.format(name1, name2, m_value)))
    
    pref.add_to(map_JPN)

# HTMLに出力
html = 'result/4_map_JPN.html'
map_JPN.save(html)
IFrame(html, width=750, height=750)
